# Task 3 - Build Department Lookup Table
Builds a table to add more descriptive text to queries in regards to department.

#### Imports

In [5]:
# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

import pandas as pd
from pandas.io import gbq

#### Connect to GBQ

In [6]:
service_path = ' '
service_file = ' ' # change this to your authentication information  
gbq_proj_id = ' ' # change this to your project. 
#gbq_dataset_id = ' ' # and change this to your data set ID
gbq_dataset_id = ' ' # and change this to your data set ID

# Creates private key
private_key =service_path + service_file


#### Credentials

In [7]:
# Get your credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And create a client to talk to GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

### Create Department Lookup Table

In [8]:
# Creates a list of department (number) and corresponding department name
department = [[1, "PACKAGED GROCERY"],
[2, "PRODUCE"],
[3, "BULK"],
[4, "REF GROCERY"],
[5, "CHEESE"],
[6, "FROZEN"],
[7, "BREAD"],
[8, "DELI"],
[9, "GEN MERCH"],
[10, "SUPPLEMENTS"],
[11, "PERSONAL CARE"],
[12, "HERBS&SPICES"],
[13, "MEAT"],
[14, "JUICE BAR"],
[15, "MISC P/I"],
[16, "FISH&SEAFOOD"],
[17, "BAKEHOUSE"],
[18, "FLOWERS"],
[19, "WEDGEWORLDWIDE"],
[20, "MISC P/I - WWW"],
[21, "CATERING"],
[22, "BEER & WINE"]
]

In [9]:
# Create the pandas DataFrame
department = pd.DataFrame(department, columns = ['deptartment', 'department_name'])

In [3]:
# Checks to make sure it worked
department

[[1, 'PACKAGED GROCERY'],
 [2, 'PRODUCE'],
 [3, 'BULK'],
 [4, 'REF GROCERY'],
 [5, 'CHEESE'],
 [6, 'FROZEN'],
 [7, 'BREAD'],
 [8, 'DELI'],
 [9, 'GEN MERCH'],
 [10, 'SUPPLEMENTS'],
 [11, 'PERSONAL CARE'],
 [12, 'HERBS&SPICES'],
 [13, 'MEAT'],
 [14, 'JUICE BAR'],
 [15, 'MISC P/I'],
 [16, 'FISH&SEAFOOD'],
 [17, 'BAKEHOUSE'],
 [18, 'FLOWERS'],
 [19, 'WEDGEWORLDWIDE'],
 [20, 'MISC P/I - WWW'],
 [21, 'CATERING'],
 [22, 'BEER & WINE']]

In [14]:
# Makes csv to upload to GBQ
department.to_csv(r'departments.csv', index = False)

### Upload to GBQ

In [16]:
my_table = "departments"

table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,my_table])

In [17]:
def tbl_exists(client, table_ref):
    from google.cloud.exceptions import NotFound
    try:
        client.get_table(table_ref)
        return True
    except NotFound:
        return False

In [19]:
if not tbl_exists(client, table_full_name) :
    table_ref = client.create_table(
        table = table_full_name
    )
else :
    table_ref = client.get_table(table_full_name)

In [20]:
table = client.get_table(table_ref)
print("Table {} contains {} columns".format(table_ref.table_id,len(table.schema)))

Table departments contains 2 columns


In [21]:
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
job_config.schema_update_options = [
    bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION # This allows us to modify the table. 
]

In [ ]:
job_config.schema = [
    bigquery.SchemaField("department", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("department_name", "STRING", mode="NULLABLE"),
 ]
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1

In [ ]:
with open("departments.csv", "rb") as source_file:
    
    job = client.load_table_from_file(
        source_file,
        table_ref,
        location="US",  # Must match the destination dataset location.
        job_config=job_config,
    )  # API request